# Mini-Competition: One-Day Start to Finish Data Science Project

## The Task:
The task is to complete a full, start-to-finish data science project in one(ish) day, using the "best practice" skills we've learned over the past day of the course. 

## The Data:
The data shows real property information, including most recent sales price as of July 2018, for properties located Washington, D.C. The goal is to predict housing prices in DC. It is from [Kaggle](https://www.kaggle.com/christophercorrea/dc-residential-properties/version/7).

### Step 1: Exploration
- Explore the data and verify it, using tests of course! 
- At least 1 visualization of the data

### Step 2: Model Selection
- MUST use gridsearchCV in a pipeline!!

### Step 3: Production
- Once you've selected your desired model, put it (and the necessary feature engineering) in a pipeline. 
- Add at least 2 tests, including one for your final output!
- Bonus points if your predictor runs as its own python program from command line (rather than in a notebook).

Metric used to judge will be MAE (mean absolute error).
The team with the lowest MAE on the holdout test set at the end of Wednesday wins!

In [1]:
cd data

C:\Users\VPL\Desktop\Data Science\DSR\DSR_Model_Pipelines_Course\data


In [2]:
from pandas.api.types import is_numeric_dtype
import ipytest.magics
import pytest
import numpy as np
# set the file name (required)
__file__ = '../Mini-Competition with Pipelines.ipynb'

import warnings
warnings.filterwarnings("ignore")


In [3]:
import pandas as pd

In [60]:
df = pd.read_csv('DC_Properties_training.csv', index_col=0, low_memory=False)
data = pd.read_csv('holdout_test_data.csv', index_col=0, low_memory=False)

In [61]:

print(df.shape)
df = df.dropna(subset=['PRICE','SALEDATE','QUADRANT','AYB'])
print(df.shape)

(157457, 48)
(97091, 48)


In [69]:
data.head()
print(data.shape)

data = data.dropna(subset=['PRICE','SALEDATE','QUADRANT','AYB'])
print(data.shape)
y_test = data['PRICE']
print(y_test)
x_test = x_test.drop('PRICE', axis=1)

(1500, 49)
(911, 49)
157458     159000.0
157460     489590.0
157461     495000.0
157462     363000.0
157463     487000.0
157464     369900.0
157465     295000.0
157468     749000.0
157471     673000.0
157473     410000.0
157474     270000.0
157475    1295000.0
157476     669900.0
157477     225000.0
157478     165500.0
157480    1170000.0
157481     140500.0
157482     265000.0
157483     335000.0
157485     560000.0
157486     265000.0
157487     160000.0
157488     458400.0
157491     610000.0
157492     501000.0
157494     230000.0
157496     605000.0
157498     187810.0
157499     715000.0
157500     725000.0
            ...    
158904      59500.0
158905     615000.0
158907     640000.0
158908     492000.0
158910     800000.0
158912     689000.0
158913     123900.0
158915      84000.0
158916     700000.0
158920     115000.0
158923     728700.0
158924     769603.0
158925     525000.0
158926     180000.0
158928     121000.0
158931      67501.0
158932     270000.0
158933     700000.0

In [6]:
df.shape

(97091, 48)

In [7]:
target = df['PRICE']
df = df.drop('PRICE', axis=1)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=10)


In [9]:
for i in df.columns:
    print(i,df[i].isnull().sum())

BATHRM 0
HF_BATHRM 0
HEAT 0
AC 0
NUM_UNITS 39937
ROOMS 0
BEDRM 0
AYB 0
YR_RMDL 40065
EYB 0
STORIES 39970
SALEDATE 0
QUALIFIED 0
SALE_NUM 0
GBA 39937
BLDG_NUM 0
STYLE 39937
STRUCT 39937
GRADE 39937
CNDTN 39937
EXTWALL 39937
ROOF 39937
INTWALL 39937
KITCHENS 39938
FIREPLACES 0
USECODE 0
LANDAREA 0
GIS_LAST_MOD_DTTM 0
SOURCE 0
CMPLX_NUM 57154
LIVING_GBA 57154
FULLADDRESS 40262
CITY 40258
STATE 40258
ZIPCODE 0
NATIONALGRID 40258
LATITUDE 0
LONGITUDE 0
ASSESSMENT_NBHD 0
ASSESSMENT_SUBNBHD 20016
CENSUS_TRACT 0
CENSUS_BLOCK 40258
WARD 0
SQUARE 0
X 0
Y 0
QUADRANT 0


In [10]:
numeric_columns = [#'NUM_UNITS',
    #'STORIES',
    #'GBA',
    #'KITCHENS',
    #'CMPLX_NUM',
    #'LIVING_GBA',
    #'SALEDATE', #take year
    'AYB',
    'CENSUS_TRACT',
    'BATHRM',
    'HF_BATHRM',
    'ROOMS',
    'BEDRM',
    'SALE_NUM',
    'LANDAREA',
    'EYB'
    ]

binary_columns = ['BLDG_NUM',
    'QUALIFIED',
    'SOURCE'
    ]

one_hot_encode_column = ['AC',
    'QUADRANT'
    #,'YR_RMDL'
    ]

label_encode_column = ['ZIPCODE',
    'HEAT',
    'USECODE'
    #,'STYLE',
    #'STRUCT',
    #'GRADE',
    #'CNDTN', #Definitely do NA-Handling
    #'EXTWALL',
    # 'ROOF',
    # 'INTWALL',
    #'ASSESSMENT_NBHD' # too many values
    #'WARD' 
    ]



In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer

class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, columns):
        self.columns = columns

    def fit(self, x, y = None):
        return self

    def transform(self, x):
        return x.loc[:,self.columns]

numeric_selector = ColumnSelector(numeric_columns)
numeric_df = numeric_selector.fit_transform(df)
print(numeric_df.head())

binary_selector = ColumnSelector(binary_columns)
binary_df = binary_selector.fit_transform(df)
print(binary_df.head())

one_hot_encode_selector = ColumnSelector(one_hot_encode_column)
one_hot_encode_df = one_hot_encode_selector.fit_transform(df)
print(one_hot_encode_df.head())

label_encode_selector = ColumnSelector(label_encode_column)
label_encode_df = label_encode_selector.fit_transform(df)

          AYB  CENSUS_TRACT  BATHRM  HF_BATHRM  ROOMS  BEDRM  SALE_NUM  \
index                                                                    
0      1910.0        4201.0       4          0      8      4         1   
2      1910.0        4201.0       3          1      9      5         3   
3      1900.0        4201.0       3          1      8      5         1   
5      1913.0        4201.0       3          2     10      5         1   
7      1906.0        4201.0       3          1      8      4         1   

       LANDAREA   EYB  
index                  
0          1680  1972  
2          1680  1984  
3          1680  1984  
5          2196  1972  
7          1627  1972  
       BLDG_NUM QUALIFIED       SOURCE
index                                 
0             1         Q  Residential
2             1         Q  Residential
3             1         Q  Residential
5             1         Q  Residential
7             1         Q  Residential
      AC QUADRANT
index            
0   

In [12]:
numeric_df.shape

(97091, 9)

# numeric columns

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(numeric_df)
print(scaled_numeric)

[[-0.96307917 -0.33063502  2.18942255 ... -0.61636411 -0.14705037
   0.22616108]
 [-0.96307917 -0.33063502  1.16690989 ...  0.81201587 -0.14705037
   0.66229189]
 [-1.23880947 -0.33063502  1.16690989 ... -0.61636411 -0.14705037
   0.66229189]
 ...
 [-0.19103434 -0.14807817 -0.87811544 ... -0.61636411 -0.583926
  -1.0095429 ]
 [-0.68734888 -0.14807817  0.14439723 ... -0.61636411 -0.59843891
  -1.66373912]
 [ 0.55343746  1.73803838 -0.87811544 ...  0.81201587 -0.64867588
  -0.02824856]]


In [14]:
%%run_pytest[clean]

def test_standardscaler():
    scaler = StandardScaler()
    scaled_numeric = scaler.fit_transform(numeric_df)
    
    assert np.isclose(scaled_numeric.mean(), 0)
    assert np.isclose(scaled_numeric.std(), 1)

============================= test session starts =============================
platform win32 -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: C:\Users\VPL\Desktop\Data Science\DSR\DSR_Model_Pipelines_Course, inifile:
plugins: palladium-1.2.0
collected 1 item

..\Mini-Competition with Pipelines.py .                                  [100%]

========================== 1 passed in 0.19 seconds ===========================


# One Hot Encoder

In [15]:
from category_encoders.one_hot import OneHotEncoder
one_hot = OneHotEncoder()
one_hot_encoded_df = one_hot.fit_transform(one_hot_encode_df.values)
print(one_hot_encode_df.columns)
print(one_hot_encoded_df.iloc[:10,:])

Index(['AC', 'QUADRANT'], dtype='object')
   0_1  0_2  0_3  0_-1  1_1  1_2  1_3  1_4  1_-1
0    1    0    0     0    1    0    0    0     0
1    1    0    0     0    1    0    0    0     0
2    1    0    0     0    1    0    0    0     0
3    1    0    0     0    1    0    0    0     0
4    1    0    0     0    1    0    0    0     0
5    1    0    0     0    1    0    0    0     0
6    1    0    0     0    1    0    0    0     0
7    1    0    0     0    1    0    0    0     0
8    1    0    0     0    1    0    0    0     0
9    1    0    0     0    1    0    0    0     0


In [16]:
%%run_pytest[clean]

def test_OneHotEncoder():
    one_hot_encoder = OneHotEncoder()
    one_hot_encoded_df = one_hot.fit_transform(one_hot_encode_df.values)
    
    # check for data leakage
    assert one_hot_encoded_df.shape[0] == one_hot_encode_df.shape[0]
    
    # check that all values have been converted into integers
    assert one_hot_encoded_df.dtypes.all() == 'int32'
    
    # check that only 0s and 1s exist in the new matrix
    assert ((one_hot_encoded_df.values ==0) | (one_hot_encoded_df.values ==1)).all()
    
    # check that a dummy column has been made for each potential category 
    assert one_hot_encoded_df.shape[1] == len(set(one_hot_encoded_df))

============================= test session starts =============================
platform win32 -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: C:\Users\VPL\Desktop\Data Science\DSR\DSR_Model_Pipelines_Course, inifile:
plugins: palladium-1.2.0
collected 1 item

..\Mini-Competition with Pipelines.py .                                  [100%]

============================== warnings summary ===============================
None
  Module already imported so cannot be rewritten: palladium

-- Docs: http://doc.pytest.org/en/latest/warnings.html
==================== 1 passed, 1 warnings in 0.28 seconds =====================


In [17]:
from category_encoders.ordinal import OrdinalEncoder
encoder = OrdinalEncoder()
label_encoded_df = encoder.fit_transform(label_encode_df.values)
print(label_encode_df.columns)
print(label_encoded_df.iloc[:10,:])
print(label_encoded_df.info())

Index(['ZIPCODE', 'HEAT', 'USECODE'], dtype='object')
         0  1   2
0  20009.0  1  24
1  20009.0  2  24
2  20009.0  2  24
3  20009.0  2  11
4  20009.0  2  24
5  20009.0  1  24
6  20009.0  1  24
7  20009.0  1  11
8  20009.0  2  24
9  20009.0  2  13
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97091 entries, 0 to 97090
Data columns (total 3 columns):
0    97091 non-null float64
1    97091 non-null int32
2    97091 non-null int64
dtypes: float64(1), int32(1), int64(1)
memory usage: 1.9 MB
None


In [18]:
all_columns = list(x_train.columns)
from sklearn.pipeline import make_pipeline, make_union

# process the numeric and categorical columns.
# then, join them all together.

processing_pipeline = make_pipeline(
    # If using make_union, then we HAVE to first select all the columns we will pull from.
    ColumnSelector(all_columns),
    make_union(
        # First, we select and 'hold out' the binary variables, as we wont do any further work to them.
        make_pipeline(
            ColumnSelector(binary_columns),
            OrdinalEncoder()
        ),
        # Pipeline for numeric features
        make_pipeline(
            ColumnSelector(numeric_columns),
            StandardScaler()
        ),
        # Pipeline for label encoded features
        make_pipeline(
            ColumnSelector(label_encode_column),
            OrdinalEncoder()
        ),
        
        # Pipeline for one-hot-encoded features
        make_pipeline(
            ColumnSelector(one_hot_encode_column),
            OneHotEncoder()
        )
    )
)

In [19]:
processed = processing_pipeline.fit_transform(df)

In [20]:
%%run_pytest[clean]

def test_processingpipeline():
    # remember, this first pipeline only acts on the features, not the target.
    processed = processing_pipeline.fit_transform(x_train)
    
    # check for data leakage
    assert x_train.shape[0] == processed.shape[0]

============================= test session starts =============================
platform win32 -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: C:\Users\VPL\Desktop\Data Science\DSR\DSR_Model_Pipelines_Course, inifile:
plugins: palladium-1.2.0
collected 1 item

..\Mini-Competition with Pipelines.py .                                  [100%]

============================== warnings summary ===============================
None
  Module already imported so cannot be rewritten: palladium

-- Docs: http://doc.pytest.org/en/latest/warnings.html
==================== 1 passed, 1 warnings in 0.64 seconds =====================


In [21]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [41]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
finalpipeline = (make_pipeline(processing_pipeline, RandomForestRegressor(n_estimators=100,max_depth=100)))
finalpipeline.fit(x_train,y_train)
y_pred = finalpipeline.predict(x_test)

print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

148912.41342407095
0.9620321249262769


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
finalpipeline = (make_pipeline(processing_pipeline, RandomForestRegressor(n_estimators=500,max_depth=100)))
finalpipeline.fit(x_train,y_train)
y_pred = finalpipeline.predict(x_test)

print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

In [22]:
finalpipeline = (make_pipeline(processing_pipeline, GradientBoostingRegressor(random_state=1, 
                                                                              n_estimators=100,
                                                                            learning_rate=0.01)))
# Fitting the pipeline
finalpipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('pipeline', Pipeline(memory=None,
     steps=[('columnselector', ColumnSelector(columns=['BATHRM', 'HF_BATHRM', 'HEAT', 'AC', 'NUM_UNITS', 'ROOMS', 'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE', 'QUALIFIED', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STRUCT', 'GRADE', 'CNDTN', 'EXTWA...e=1, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False))])

In [70]:

y_pred = finalpipeline.predict(x_test)

In [71]:
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

166834.11284511906
0.9915624964128249


In [33]:
# Number of trees in gradient Boosting
n_estimators = [500]
# Number of features to consider at every split
learning_rate = [0.001, 0.01]


# Create the random grid
random_grid = {'gradientboostingregressor__n_estimators': n_estimators,
               'gradientboostingregressor__learning_rate': learning_rate
              }

print(random_grid)

print("Grid search")
print('\n')

params = random_grid
scoring = 'neg_mean_absolute_error'
clf = GridSearchCV(finalpipeline, params, n_jobs=1, verbose=True, scoring=scoring)
clf.fit(df, target)

{'gradientboostingregressor__n_estimators': [500], 'gradientboostingregressor__learning_rate': [0.001, 0.01]}
Grid search


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.8min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pipeline', Pipeline(memory=None,
     steps=[('columnselector', ColumnSelector(columns=['BATHRM', 'HF_BATHRM', 'HEAT', 'AC', 'NUM_UNITS', 'ROOMS', 'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE', 'QUALIFIED', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STRUCT', 'GRADE', 'CNDTN', 'EXTWA...e=1, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'gradientboostingregressor__n_estimators': [500], 'gradientboostingregressor__learning_rate': [0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=True)

In [34]:
all_params = (clf.cv_results_)
all_params = pd.DataFrame(all_params)
all_params.sort_values('rank_test_score').head(10)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_gradientboostingregressor__learning_rate,param_gradientboostingregressor__n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,28.309447,0.385606,-808855.848041,-497338.735039,0.01,500,{'gradientboostingregressor__learning_rate': 0...,1,-589865.845407,-667205.365987,-439553.029333,-6.437420e+05,-1.397167e+06,-181068.870723,2.652448,0.032918,420494.072129,223841.615229
0,25.286955,0.397253,-867309.210513,-763347.895811,0.001,500,{'gradientboostingregressor__learning_rate': 0...,2,-441904.314108,-981308.901312,-710971.535630,-1.014619e+06,-1.449070e+06,-294115.405980,2.186811,0.032735,425774.303383,332076.039621


In [27]:
gridsearch_predictions = compare_predictions(df, target, clf, mean_absenteeism_time)

NameError: name 'compare_predictions' is not defined